# Chapter 5. Deep Learning Computation

## Layer & Blocks
- Single neuron
  - 여러개의 입력이 있고
  - 하나의 Scalar 출력을 갖고
  - Paramter들을 가짐 (w.....,b)
- Layer
  - has Set of Input
  - generate corresponding output
  - parameter들을 가짐 
- Model 
  - Layer와 동일한 특징을 가짐 (Set of Input / Corresponding Output / Tunaable Paramter)
> MLP의 경우 Model과 Layer는 이처럼 공통적인 특징 그리고 구조를 가지고 있음

- block
  - 단일 layer 혹은 group of layer 혹은 전체 Model 일수 있음
  - 여러개의 block을 조합하여 더 복잡한 모델을 만들 수 있음
  - 프로그램 관점에서 보면 class 임
  
 

  

In [12]:
import torch
from torch import nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.relu = nn.ReLU()
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(self.relu(self.hidden(X)))
    
X = torch.normal(0, std=0.01, size=(2, 20), dtype=torch.float32)

net = MLP()
net(X)


tensor([[ 0.1549,  0.0502,  0.0058,  0.0386,  0.0868,  0.0649, -0.0149, -0.0220,
         -0.0678,  0.0866],
        [ 0.1514,  0.0507,  0.0057,  0.0409,  0.0848,  0.0598, -0.0138, -0.0212,
         -0.0703,  0.0874]], grad_fn=<AddmmBackward0>)

- nn.Module class는 _modules 속성을 지니고 있으며 framework은 parameter 초기화 시 이 _modules 속성을 검사하여 재귀적으로 paramter 초기화를 수행한다.  
  - nn.Module
    - _modulues
      - nn.Module
        - _modules ....
        

In [109]:
import torch
from torch import nn

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256,10))
net(X)
        
        

block


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x4 and 20x256)

## Parameter Management
- 모델로 부터 parameter를 추출하고 이를 이용하여 진단, 시각화 하기
- paramter 초기화
- 서로 다른 모델 요소간의 parameter 교환

### Accessing Parameter from Model - state_dict()
- weight과 bias의 정보를 포함
- float32 형식
- index를 이용하여 특정 layer를 지정할 수 있음
- Parameter class의 instance
- grad 속성을 가지고 있음. 하지만 backward를 하지 않았기 때문에 None 임

In [48]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,1))
X = torch.normal(0, std=0.1, size=(2,4))
net(X)

print(net[2].state_dict())
print(net[2].weight)
print(type(net[2].weight))
print(net[2].weight.data)
print(net[2].weight.grad)

OrderedDict([('weight', tensor([[-0.0106, -0.0027,  0.1652, -0.0301,  0.2782,  0.2915,  0.3414, -0.1533]])), ('bias', tensor([-0.0782]))])
Parameter containing:
tensor([[-0.0106, -0.0027,  0.1652, -0.0301,  0.2782,  0.2915,  0.3414, -0.1533]],
       requires_grad=True)
<class 'torch.nn.parameter.Parameter'>
tensor([[-0.0106, -0.0027,  0.1652, -0.0301,  0.2782,  0.2915,  0.3414, -0.1533]])
None


In [40]:
print(*[(name, param.size) for name, param in net[0].named_parameters()])


('weight', <built-in method size of Parameter object at 0x7f2bb3a15310>) ('bias', <built-in method size of Parameter object at 0x7f2bb3a152c0>)


### Access Parameters from Nested Model

In [54]:
import torch
from torch import nn

def model1():
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(), 
                         nn.Linear(8,4), nn.ReLU())

def model2():
    net = nn.Sequential()
    for i in range(5):
        net.add_module(f'{i}',model1())
    return net

rgnet = nn.Sequential(model2(), nn.Linear(4,1))
print(rgnet(X))
print(rgnet)
print(rgnet[0][0][0].weight.data)



tensor([[0.2039],
        [0.2039]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (4): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU

### Paramter Initialization
#### Built-in Initialization
- zero / normal / const ... 다앙햔 pre-defined initializer가 제공됨


In [71]:
import torch
from torch import nn

def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
        nn.init.zeros_(m.bias)

def init_const(m):   
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,1) 
        nn.init.zeros_(m.bias)

def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)

def init_custom(m):
    if type(m) == nn.Linear:
        m.weight.data = torch.rand(size=m.weight.data.shape)
        m.weight.data += 10
        
net.apply(init_const)
print(*[net[0].weight.data[0], net[0].bias.data[0]])

net[0].apply(init_xavier)
net[2].apply(init_custom)
print(net[0].weight.data)
print(net[2].weight.data)


tensor([1., 1., 1., 1.]) tensor(0.)
tensor([[-1.8210e-01,  1.7192e-01,  2.7038e-01, -9.0876e-01],
        [-3.6169e-01,  5.4119e-01, -3.3802e-01, -6.8608e-01],
        [ 2.4508e-01, -1.3278e-01, -1.3855e-01, -7.7812e-04],
        [ 3.6397e-03, -3.9097e-01, -2.2136e-01, -2.9122e-01],
        [-7.2229e-01,  3.1027e-01,  1.7012e-02,  3.7291e-01],
        [-6.6274e-01, -5.8979e-01, -3.3946e-01, -2.6099e-01],
        [-9.0365e-03, -9.6022e-02, -6.1467e-01, -4.1452e-01],
        [-1.8167e-01, -3.8665e-01, -4.8293e-01, -4.8055e-01]])
tensor([[10.4461, 10.4819, 10.8835, 10.9754, 10.4175, 10.1001, 10.9889, 10.8458]])


#### Tied Parameters 
- 서로 다른 Layer들 간에 paramter를 공유하도록 할 수 있음 
- paramter가 공유된다기 보다는 실제로는 하나의 Layer를 여러 layer에서 참조하는 느낌


In [76]:
import torch
from torch import nn

shared = nn.Linear(8,8)

net = nn.Sequential(nn.Linear(4,8), nn.ReLU(),
                   shared, nn.ReLU(),
                   shared, nn.ReLU(),
                   nn.Linear(8,1))

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0]) # 1. value of parameters should be same 
net[2].weight.data[0][0] = 100                        # 2. modification of paramters of shared 
                                                      # paramter is visible from the others
print(net[2].weight.data[0] == net[4].weight.data[0]) 
print(net[2].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
tensor([ 1.0000e+02, -2.7184e-02, -2.6191e-01, -1.8853e-01, -1.5053e-01,
         1.8554e-01, -2.5032e-01, -3.1696e-01])


## Custom Layers
### Layers without parameters
- parameter가 없는 임의의 layer를 nn.Module을 상속하여 만들 수 있음 
- 이렇게 만들어진 custom layers는 다른 layer들과 함께 block처럼 modular하게 사용될 수 있음


In [80]:
import torch
from torch import nn

class CenteredLayer(nn.Module):  # element wise mean 값을 빼주는 custom layer
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()
    
    
layer = CenteredLayer()

net = nn.Sequential(nn.Linear(4, 8), CenteredLayer())

Y = net(X)   
Y.mean()   # 결과물의 mean이 0 인지(매우 가까운지 확인)


tensor(9.3132e-09, grad_fn=<MeanBackward0>)

### Layers with Parameters (Linear Layer Internal)


In [93]:
import torch
from torch import nn

class MyLinear(nn.Module):
    def __init__(self, in_count, out_count):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(size=(in_count, out_count), dtype=torch.float32))
        self.bias = nn.Parameter(torch.tensor(0, dtype=torch.float32))
    
    def forward(self, X):
        return torch.matmul(X, self.weight) + self.bias

net = nn.Sequential(MyLinear(4,8), nn.ReLU())
net(X)
print(net[0].weight.data)

tensor([[0.4483, 0.1872, 0.9252, 0.2744, 0.6789, 0.8821, 0.5642, 0.8725],
        [0.6620, 0.9309, 0.1771, 0.3165, 0.0416, 0.7485, 0.8869, 0.7954],
        [0.7602, 0.9064, 0.8001, 0.6923, 0.8938, 0.5408, 0.8832, 0.1485],
        [0.9549, 0.1790, 0.8704, 0.5390, 0.7758, 0.2422, 0.7196, 0.1501]])


## File I/O
- 학습된 모델의 저장 및 불러오기
- 아주 오랜 기간 동안의 training을 진행할 경우 주기적으로 모델을 저장하는 것이 최선

### Loading and Saving Tensors
- tensors들을 저장할 때 torch에서 제공되는 ```save()``` 그리고 ```load()``` 함수를 사용
- 저장 시 이름을 아래와 같이 지정하여 복원시에 사용


In [96]:
import torch 
from torch import nn

xx = torch.rand(size=(2,4))
torch.save(xx, 'file-x')
print(xx)
xxx = torch.load('file-x')
print(xxx)

tensor([[0.7295, 0.3096, 0.3754, 0.5856],
        [0.0182, 0.3196, 0.7680, 0.8578]])
tensor([[0.7295, 0.3096, 0.3754, 0.5856],
        [0.0182, 0.3196, 0.7680, 0.8578]])


In [119]:
import torch
from torch import nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        for i in range(5):
            self._modules[str(i)] = nn.Sequential(nn.Linear(4,4), nn.ReLU())
        
    def forward(self, X):
        for model in self._modules.values():
            X = model(X)
        return X

net = MLP()
torch.save(net, 'model')  # save model as a whole

net2 = torch.load('model') # load model 
net2(X)


torch.save(net.state_dict(), 'model_param')  # save only parameters of the model

clone = MLP()
clone.load_state_dict(torch.load('model_param')) # load saved parameters into empty model

clone(X)



tensor([[0.0000, 0.0000, 0.3573, 0.2071],
        [0.0000, 0.0000, 0.3579, 0.2082]], grad_fn=<ReluBackward0>)

### Saving Whole Model vs. Saving model paramters using ```state_dict()```
- ```torch.save()```를 이용하여 model을 저장하면 pickle 직렬화를 이용하게되는데, 이때 class에 대한 정의가 필요하며 해당 class 정의에 대한 참조 (directory path)를 저장한다. 따라서 다른 환경에서 모델을 불러 올경우 이러한 경로의 차이에 의해서 깨질 수 있다.
- ```state_dict()```를 사용하는 것이 권장됨

## GPU
### Computing Devices
- ```torch.device()```를 이용하여 저장 및 연산 장치를 선택할 수 있다. 
- Default로 CPU로 지정된다.


In [130]:
import torch
from torch import nn

!nvidia-smi
print(*[torch.device('cuda'), torch.device('cpu')])
torch.cuda.device_count()

Sun Apr 17 13:22:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 30%   33C    P8    16W / 130W |    364MiB /  2048MiB |     30%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

1

In [136]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

devices = try_all_gpus()
print(devices)
    
        

[device(type='cuda', index=0)]


### Tensors and GPUs
- Default로 모든 tensor는 CPU의 memory에 생성된다.
- tensor가 생성 될 device를 지정할 수 있다.

In [149]:
import torch

yy = torch.tensor([2,4], dtype=torch.float32)
print(yy.device) # tensor created @ CPU

xx = torch.tensor([2,4], dtype=torch.float32, device=try_gpu())
print(xx.device) # tensor created @ GPU if you have one

xx

yy.cuda(0) is yy

net.to(try_gpu())
net(torch.rand(size=(4,4), device=try_gpu()))

cpu
cuda:0


tensor([[0.0000, 0.0000, 0.3574, 0.2090],
        [0.0000, 0.0000, 0.3574, 0.2099],
        [0.0000, 0.0000, 0.3574, 0.2109],
        [0.0000, 0.0000, 0.3574, 0.2092]], device='cuda:0',
       grad_fn=<ReluBackward0>)